# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import time
import numpy as np
import pandas as pd
import sqlalchemy as db
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
from sklearn.svm import SVC
from sklearn.externals import joblib 
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
# load data from database
engine = create_engine('sqlite:///Fig8_Messages.db')
df = pd.read_sql_table('Fig8_Messages', engine)
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df.message.values
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1).values

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidif', TfidfTransformer()),
    ('multiclf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [8]:
# Timing how long to train using pipeline
start = time.time()

pipeline.fit(X_train, y_train)

end = time.time()
print("Time elapsed:", (end - start) / 60, "minutes")

Time elapsed: 1.2532131870587666 minutes


In [9]:
y_preds = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
accuracy = (y_preds == y_test).mean()
print("Accuracy:", accuracy)

Accuracy: 0.941193673075


In [11]:
target_names = list(df.drop(['id', 'message', 'original', 'genre'], axis=1))
#target_names

In [12]:
for i in range(len(target_names)):
    print("Column:", target_names[i])
    print(classification_report(y_test[:,i], y_preds[:,i]))
    print("----------------------------------------------------")

Column: related
             precision    recall  f1-score   support

          0       0.62      0.37      0.46      1536
          1       0.82      0.93      0.87      4977
          2       1.00      0.10      0.18        41

avg / total       0.78      0.79      0.77      6554

----------------------------------------------------
Column: request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5418
          1       0.80      0.37      0.50      1136

avg / total       0.87      0.87      0.85      6554

----------------------------------------------------
Column: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

----------------------------------------------------
Column: aid_related
             precision    recall  f1-score   support

          0       0.71 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
#print(pipeline.get_params().keys())

In [13]:
parameters = {
    'vect__ngram_range': [(1,1), (1,2)],
    'multiclf__estimator__n_estimators': [50, 100, 150],
}

cv = GridSearchCV(estimator=pipeline, 
                  param_grid=parameters, 
                  cv=5, n_jobs=-1)

In [ ]:
# Timing how long to train using GridSearch
start = time.time()

cv.fit(X_train, y_train)

end = time.time()
print("Time elapsed:", (end - start) / 60, "minutes")

In [ ]:
y_preds_cv = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
accuracy = (y_preds_cv == y_test).mean()
print("Accuracy:", accuracy)

In [ ]:
for i in range(len(target_names)):
    print("Column:", target_names[i])
    print(classification_report(y_test[:,i], y_preds_cv[:,i]))
    print("----------------------------------------------------")

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidif', TfidfTransformer()),
    ('svc_clf', MultiOutputClassifier(OneVsRestClassifier(SVC())))
])

In [ ]:
# Timing how long to train using pipeline
start = time.time()

pipeline2.fit(X_train, y_train)

end = time.time()
print("Time elapsed:", (end - start) / 60, "minutes")

In [ ]:
y_preds_SVC = pipeline2.predict(X_train)

In [ ]:
# accuracy = (y_preds_SVC == y_test).mean()
# print("Accuracy:", accuracy)

### 9. Export your model as a pickle file

In [ ]:
# Save the GridSearch model
joblib.dump(cv, 'classifier.pkl') 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.